In [ ]:
import csv
import pandas as pd 
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm
import pyodbc

In [ ]:
pkdf = pd.DataFrame()
ppdf = pd.DataFrame()
esdf = pd.DataFrame()

single_date = date.today()
single_date = single_date.strftime("%Y-%m-%d")

#single_date = '2021-10-30'
year = single_date[0:4]
month = single_date[5:7]
day = single_date[8:10]

session = requests.Session()
retry = Retry(connect=30, backoff_factor=0.1)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

pkurl = 'https://www.naturalstattrick.com/teamtable.php?fromseason=20212022&thruseason=20212022&stype=2&sit=pk&score=all&rate=y&team=all&loc=B&gpf=410&fd='+single_date+'&td='+single_date
ppurl = 'https://www.naturalstattrick.com/teamtable.php?fromseason=20212022&thruseason=20212022&stype=2&sit=pp&score=all&rate=y&team=all&loc=B&gpf=410&fd='+single_date+'&td='+single_date
esurl = 'https://www.naturalstattrick.com/teamtable.php?fromseason=20212022&thruseason=20212022&stype=2&sit=ev&score=all&rate=y&team=all&loc=B&gpf=410&fd='+single_date+'&td='+single_date

#penalty kill
#'https://www.naturalstattrick.com/teamtable.php?fromseason=20172018&thruseason=20172018&stype=2&sit=pk&score=all&rate=y&team=all&loc=B&gpf=410&fd='+single_date+'&td='+single_date
#power play
#'https://www.naturalstattrick.com/teamtable.php?fromseason=20172018&thruseason=20172018&stype=2&sit=pp&score=all&rate=y&team=all&loc=B&gpf=410&fd='+single_date+'&td='+single_date
#evenstrength
#'https://www.naturalstattrick.com/teamtable.php?fromseason=20172018&thruseason=20172018&stype=2&sit=ev&score=all&rate=y&team=all&loc=B&gpf=410&fd='+single_date+'&td='+single_date

# Penalty Kill Url to dataframe
pkhtml = requests.get(pkurl).content
pkdf_list = pd.read_html(pkhtml)
pkdf = pkdf_list[-1]
pkdf.insert(0, 'Date', single_date)

time.sleep(60)
# Power Play Url to dataframe
pphtml = requests.get(ppurl).content
ppdf_list = pd.read_html(pphtml)
ppdf = ppdf_list[-1]
ppdf.insert(0, 'Date', single_date)

time.sleep(60)
# Even Strength Url to dataframe
eshtml = requests.get(esurl).content
esdf_list = pd.read_html(eshtml)
esdf = esdf_list[-1]
esdf.insert(0, 'Date', single_date)

#Print confirmation of Date
print('url '+single_date)

# Dataframe Edits 
#Delete Unneccesary Data Column Unnamed: 0
del pkdf['Unnamed: 0']
del ppdf['Unnamed: 0']
del esdf['Unnamed: 0']

#Insert new columns 
# Year
pkdf.insert(0, "Year", year, True)
ppdf.insert(0, "Year", year, True)
esdf.insert(0, "Year", year, True)

# Month
pkdf.insert(0, "Month", month, True)
ppdf.insert(0, "Month", month, True)
esdf.insert(0, "Month", month, True)

# Day
pkdf.insert(0, "Day",day, True)
ppdf.insert(0, "Day",day, True)
esdf.insert(0, "Day",day, True)

#Opponent  
#Even strength can use normal flipping of previous/succeeding row of that team
esdf.insert(0, 'Opponent', '')
for i in range(0, len(esdf)):
    if i == 0:
        esdf.loc[i, 'Opponent'] = esdf.loc[i+1, 'Team']  
    elif i % 2:
        esdf.loc[i, 'Opponent'] = esdf.loc[i-1, 'Team'] 
    else:
        esdf.loc[i, 'Opponent'] = esdf.loc[i+1, 'Team'] 

#Penalty Kill and Power Play have to use a dictionary based off of Even Strength 
#This is because not all teams have a penalty kill/power play during a game 
Opponentdict = dict(zip(esdf.Team, esdf.Opponent))  

ppdf.insert(0, 'Opponent', ppdf['Team'])
ppdf['Opponent'] = ppdf['Opponent'].map(Opponentdict)

pkdf.insert(0, 'Opponent', pkdf['Team']) 
pkdf['Opponent'] = pkdf['Opponent'].map(Opponentdict)

###
#Location
###
esdf.insert(0, 'Location', esdf['Team']) 
esdf['Location'] =np.where(esdf.index%2==0, 'Away', 'Home')

Locationdict = dict(zip(esdf.Team, esdf.Location)) 

ppdf.insert(0, 'Location', ppdf['Team']) 
ppdf['Location'] = ppdf['Location'].map(Locationdict)

pkdf.insert(0, 'Location', pkdf['Team']) 
pkdf['Location'] = pkdf['Location'].map(Locationdict)

###
#GameID
###
Monthdict = {'1': 'Jan', '2': 'Feb', '3': 'Mar', '4': 'Apr', '5': 'May', '6': 'Jun', '7': 'Jul', '8': 'Aug',
           '9': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}
sGameID = pkdf['Day']+'-'+ pkdf['Month'].map(Monthdict)+'-'+pkdf['Year']+' '+ pkdf['Team']
pkdf.insert(0, 'GameID', sGameID)

sGameID = ppdf['Day']+'-'+ ppdf['Month'].map(Monthdict)+'-'+ppdf['Year']+' '+ ppdf['Team']
ppdf.insert(0, 'GameID', sGameID)

sGameID = esdf['Day']+'-'+ esdf['Month'].map(Monthdict)+'-'+esdf['Year']+' '+ esdf['Team']
esdf.insert(0, 'GameID', sGameID)

# Replace - and NaN with None so accepted by SQL command 
###
# Penalty Kill
###
pkdf.iloc[:, 7:78] = pkdf.iloc[:, 7:78].replace('-','NaN')
pkdf = pkdf.where(pd.notnull(pkdf), None)
###
# Power Play
###
ppdf.iloc[:, 7:78] = ppdf.iloc[:, 7:78].replace('-','NaN')
ppdf = ppdf.where(pd.notnull(ppdf), None)
###
# Even Strength 
###
esdf.iloc[:, 7:78] = esdf.iloc[:, 7:78].replace('-','NaN')
esdf = esdf.where(pd.notnull(esdf), None)

In [ ]:
###
#Penalty Kill
###
# server = 'Server=DESKTOP-TS4R5T7\SQLEXPRESS' 
# database = 'NHL' 
# username = '' 
# password = '' 
# cnxn = pyodbc.connect('Driver={SQL Server};'
#                       'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
#                       'Database=NHL;'
#                       'Trusted_Connection=yes;')

# cursor = cnxn.cursor()
# # # Insert Dataframe into SQL Server:
# for index, row in pkdf.iterrows():
#      cursor.execute("INSERT INTO  [NHL].[dbo].[Master Penalty Kill](GameID, Location, Opponent, Day, Month, Year, Date, Team, GP, TOI_GP, W, L, OTL, ROW, Points, Point, CF_60, CA_60, CF, FF_60, FA_60, FF, SF_60, SA_60, SF, GF_60, GA_60, GF, xGF_60, xGA_60, xGF, SCF_60, SCA_60, SCF, SCSF_60, SCSA_60, SCSF, SCGF_60, SCGA_60, SCGF, SCSH, SCSV, HDCF_60, HDCA_60, HDCF, HDSF_60, HDSA_60, HDSF, HDGF_60, HDGA_60, HDGF, HDSH, HDSV, MDCF_60, MDCA_60, MDCF, MDSF_60, MDSA_60, MDSF, MDGF_60, MDGA_60, MDGF, MDSH, MDSV, LDCF_60, LDCA_60, LDCF, LDSF_60, LDSA_60, LDSF, LDGF_60, LDGA_60, LDGF, LDSH, LDSV, SH, SV, PDO) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", 
#                    row['GameID'],row['Location'],row['Opponent'],row['Day'],row['Month'],row['Year'],row['Date'],row['Team'],row['GP'],row['TOI/GP'],row['W'],row['L'],row['OTL'],row['ROW'],row['Points'],row['Point %'],row['CF/60'],row['CA/60'],row['CF%'],row['FF/60'],row['FA/60'],row['FF%'],row['SF/60'],row['SA/60'],row['SF%'],row['GF/60'],row['GA/60'],row['GF%'],row['xGF/60'],row['xGA/60'],row['xGF%'],row['SCF/60'],row['SCA/60'],row['SCF%'],row['SCSF/60'],row['SCSA/60'],row['SCSF%'],row['SCGF/60'],row['SCGA/60'],row['SCGF%'],row['SCSH%'],row['SCSV%'],row['HDCF/60'],row['HDCA/60'],row['HDCF%'],row['HDSF/60'],row['HDSA/60'],row['HDSF%'],row['HDGF/60'],row['HDGA/60'],row['HDGF%'],row['HDSH%'],row['HDSV%'],row['MDCF/60'],row['MDCA/60'],row['MDCF%'],row['MDSF/60'],row['MDSA/60'],row['MDSF%'],row['MDGF/60'],row['MDGA/60'],row['MDGF%'],row['MDSH%'],row['MDSV%'],row['LDCF/60'],row['LDCA/60'],row['LDCF%'],row['LDSF/60'],row['LDSA/60'],row['LDSF%'],row['LDGF/60'],row['LDGA/60'],row['LDGF%'],row['LDSH%'],row['LDSV%'],row['SH%'],row['SV%'],row['PDO'])

# cnxn.commit()
# cursor.close()

# ###
# # Power Play
# ###
# server = 'Server=DESKTOP-TS4R5T7\SQLEXPRESS' 
# database = 'NHL' 
# username = '' 
# password = '' 
# cnxn = pyodbc.connect('Driver={SQL Server};'
#                       'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
#                       'Database=NHL;'
#                       'Trusted_Connection=yes;')

# cursor = cnxn.cursor()
# # # Insert Dataframe into SQL Server:
# for index, row in ppdf.iterrows():
#      cursor.execute("INSERT INTO  [NHL].[dbo].[Master Penalty Kill](GameID, Location, Opponent, Day, Month, Year, Date, Team, GP, TOI_GP, W, L, OTL, ROW, Points, Point, CF_60, CA_60, CF, FF_60, FA_60, FF, SF_60, SA_60, SF, GF_60, GA_60, GF, xGF_60, xGA_60, xGF, SCF_60, SCA_60, SCF, SCSF_60, SCSA_60, SCSF, SCGF_60, SCGA_60, SCGF, SCSH, SCSV, HDCF_60, HDCA_60, HDCF, HDSF_60, HDSA_60, HDSF, HDGF_60, HDGA_60, HDGF, HDSH, HDSV, MDCF_60, MDCA_60, MDCF, MDSF_60, MDSA_60, MDSF, MDGF_60, MDGA_60, MDGF, MDSH, MDSV, LDCF_60, LDCA_60, LDCF, LDSF_60, LDSA_60, LDSF, LDGF_60, LDGA_60, LDGF, LDSH, LDSV, SH, SV, PDO) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", 
#                    row['GameID'],row['Location'],row['Opponent'],row['Day'],row['Month'],row['Year'],row['Date'],row['Team'],row['GP'],row['TOI/GP'],row['W'],row['L'],row['OTL'],row['ROW'],row['Points'],row['Point %'],row['CF/60'],row['CA/60'],row['CF%'],row['FF/60'],row['FA/60'],row['FF%'],row['SF/60'],row['SA/60'],row['SF%'],row['GF/60'],row['GA/60'],row['GF%'],row['xGF/60'],row['xGA/60'],row['xGF%'],row['SCF/60'],row['SCA/60'],row['SCF%'],row['SCSF/60'],row['SCSA/60'],row['SCSF%'],row['SCGF/60'],row['SCGA/60'],row['SCGF%'],row['SCSH%'],row['SCSV%'],row['HDCF/60'],row['HDCA/60'],row['HDCF%'],row['HDSF/60'],row['HDSA/60'],row['HDSF%'],row['HDGF/60'],row['HDGA/60'],row['HDGF%'],row['HDSH%'],row['HDSV%'],row['MDCF/60'],row['MDCA/60'],row['MDCF%'],row['MDSF/60'],row['MDSA/60'],row['MDSF%'],row['MDGF/60'],row['MDGA/60'],row['MDGF%'],row['MDSH%'],row['MDSV%'],row['LDCF/60'],row['LDCA/60'],row['LDCF%'],row['LDSF/60'],row['LDSA/60'],row['LDSF%'],row['LDGF/60'],row['LDGA/60'],row['LDGF%'],row['LDSH%'],row['LDSV%'],row['SH%'],row['SV%'],row['PDO'])

# cnxn.commit()
# cursor.close()

# ###
# #Even Strength 
# ###
# server = 'Server=DESKTOP-TS4R5T7\SQLEXPRESS' 
# database = 'NHL' 
# username = '' 
# password = '' 
# cnxn = pyodbc.connect('Driver={SQL Server};'
#                       'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
#                       'Database=NHL;'
#                       'Trusted_Connection=yes;')

# cursor = cnxn.cursor()
# # # Insert Dataframe into SQL Server:
# for index, row in esdf.iterrows():
#      cursor.execute("INSERT INTO  [NHL].[dbo].[Master Penalty Kill](GameID, Location, Opponent, Day, Month, Year, Date, Team, GP, TOI_GP, W, L, OTL, ROW, Points, Point, CF_60, CA_60, CF, FF_60, FA_60, FF, SF_60, SA_60, SF, GF_60, GA_60, GF, xGF_60, xGA_60, xGF, SCF_60, SCA_60, SCF, SCSF_60, SCSA_60, SCSF, SCGF_60, SCGA_60, SCGF, SCSH, SCSV, HDCF_60, HDCA_60, HDCF, HDSF_60, HDSA_60, HDSF, HDGF_60, HDGA_60, HDGF, HDSH, HDSV, MDCF_60, MDCA_60, MDCF, MDSF_60, MDSA_60, MDSF, MDGF_60, MDGA_60, MDGF, MDSH, MDSV, LDCF_60, LDCA_60, LDCF, LDSF_60, LDSA_60, LDSF, LDGF_60, LDGA_60, LDGF, LDSH, LDSV, SH, SV, PDO) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", 
#                    row['GameID'],row['Location'],row['Opponent'],row['Day'],row['Month'],row['Year'],row['Date'],row['Team'],row['GP'],row['TOI/GP'],row['W'],row['L'],row['OTL'],row['ROW'],row['Points'],row['Point %'],row['CF/60'],row['CA/60'],row['CF%'],row['FF/60'],row['FA/60'],row['FF%'],row['SF/60'],row['SA/60'],row['SF%'],row['GF/60'],row['GA/60'],row['GF%'],row['xGF/60'],row['xGA/60'],row['xGF%'],row['SCF/60'],row['SCA/60'],row['SCF%'],row['SCSF/60'],row['SCSA/60'],row['SCSF%'],row['SCGF/60'],row['SCGA/60'],row['SCGF%'],row['SCSH%'],row['SCSV%'],row['HDCF/60'],row['HDCA/60'],row['HDCF%'],row['HDSF/60'],row['HDSA/60'],row['HDSF%'],row['HDGF/60'],row['HDGA/60'],row['HDGF%'],row['HDSH%'],row['HDSV%'],row['MDCF/60'],row['MDCA/60'],row['MDCF%'],row['MDSF/60'],row['MDSA/60'],row['MDSF%'],row['MDGF/60'],row['MDGA/60'],row['MDGF%'],row['MDSH%'],row['MDSV%'],row['LDCF/60'],row['LDCA/60'],row['LDCF%'],row['LDSF/60'],row['LDSA/60'],row['LDSF%'],row['LDGF/60'],row['LDGA/60'],row['LDGF%'],row['LDSH%'],row['LDSV%'],row['SH%'],row['SV%'],row['PDO'])

# cnxn.commit()
# cursor.close()


In [5]:
import os
os.getcwd()


'C:\\Users\\michi'